In [145]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,Dataset
from tqdm.notebook import tqdm # Progession bar
from PIL import Image


In [146]:

src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)

data_dir = os.path.join('..', 'data')
metadata_dir = os.path.join(data_dir, 'metadata')
test_csv = os.path.join(metadata_dir, 'test_metadata.csv')
train_csv = os.path.join(metadata_dir, 'train_metadata.csv')
val_csv = os.path.join(metadata_dir, 'val_metadata.csv')

images_dir = os.path.join(data_dir,'images')
processed_dir = os.path.join(data_dir,'processed')
model_dir = os.path.join('..','models','deepcluster')

In [147]:
def make_data_loaders_unsupervised(train_csv, val_csv, test_csv,processed_dir, images_dir, batch_size, image_size):
    from torch.utils.data import DataLoader, Dataset
    from PIL import Image

    class ImageDataset(Dataset):
        def __init__(self, csv_file, root_dir, transform=None):
            self.data = pd.read_csv(csv_file)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
            image = Image.open(img_name).convert('RGB')

            # Obtener las etiquetas (asumiendo que empiezan desde el segundo índice)
            labels = self.data.iloc[idx, 1:].values.astype(np.float32)

            if self.transform:
                image = self.transform(image)

            return image, labels

    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageDataset(train_csv, processed_dir, transform)
    val_dataset = ImageDataset(val_csv, images_dir, transform)
    test_dataset = ImageDataset(test_csv, images_dir, transform)

    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
        'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
        'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    }
    return dataloaders

In [148]:

train_csv = os.path.join(metadata_dir,'train_metadata.csv')
val_csv =os.path.join(metadata_dir,'val_metadata.csv')
test_csv =val_csv
dataloaders = make_data_loaders_unsupervised(train_csv,val_csv,test_csv,processed_dir,images_dir,33,224)
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [149]:
class DeepClusterModel(nn.Module):
    def __init__(self, base_model):
        super(DeepClusterModel, self).__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Linear(base_model.fc.in_features, 128)  

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [150]:

from torchvision.models import ResNet50_Weights
def initialize_model():
    base_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    model = DeepClusterModel(base_model)
    return model

In [151]:
from sklearn.cluster import KMeans
def run_kmeans(features, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, n_init=20, verbose=1)
    assignments = kmeans.fit_predict(features)
    return assignments

In [152]:
def train_deepcluster(model, train_dataloader,val_loader, num_clusters, num_epochs):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        model.train()
        features = []
        print(features)
        for img,_ in train_dataloader:
           
            inputs = img.cuda()
            outputs = model(inputs)
           
            features.append(outputs.detach().cpu().numpy())

        features = np.vstack(features)
        assignments = run_kmeans(features, num_clusters)

        model.train()
        train_loss = 0.0
        for batch_idx, (img, _) in enumerate(dataloaders['train']):
            inputs = img.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            batch_assignments = torch.tensor(assignments[batch_idx * inputs.size(0):(batch_idx + 1) * inputs.size(0)], dtype=torch.long).cuda()
            loss = criterion(outputs, batch_assignments)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_dataloader))

        # Calcular la pérdida de validación
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for img,_ in val_loader:
                inputs = img.cuda()
                outputs = model(inputs)
                batch_assignments = torch.tensor(assignments[:inputs.size(0)], dtype=torch.long).cuda()
                loss = criterion(outputs, batch_assignments)
                val_loss += loss.item()
        
        val_losses.append(val_loss / len(val_loader))

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}")

    return train_losses, val_losses
        




In [153]:
for i in dataloaders['train']:
    print(i)
    break

for i in dataloaders['val']:
    print(i)
    break

[tensor([[[[-1.9980, -1.9980, -2.0152,  ..., -1.9809, -1.9809, -1.9809],
          [-1.9980, -1.9980, -2.0152,  ..., -1.9809, -1.9809, -1.9809],
          [-1.9980, -1.9980, -2.0152,  ..., -1.9809, -1.9809, -1.9809],
          ...,
          [ 0.0912,  0.0227, -0.0629,  ..., -0.3712, -0.2171, -0.1143],
          [ 0.0912,  0.0227, -0.0629,  ..., -0.3712, -0.2171, -0.1143],
          [ 0.0912,  0.0227, -0.0629,  ..., -0.3712, -0.2171, -0.1143]],

         [[-1.9132, -1.9132, -1.9307,  ..., -1.8957, -1.8957, -1.8957],
          [-1.9132, -1.9132, -1.9307,  ..., -1.8957, -1.8957, -1.8957],
          [-1.9132, -1.9132, -1.9307,  ..., -1.8957, -1.8957, -1.8957],
          ...,
          [ 0.2227,  0.1527,  0.0651,  ..., -0.2500, -0.0924,  0.0126],
          [ 0.2227,  0.1527,  0.0651,  ..., -0.2500, -0.0924,  0.0126],
          [ 0.2227,  0.1527,  0.0651,  ..., -0.2500, -0.0924,  0.0126]],

         [[-1.6824, -1.6824, -1.6999,  ..., -1.6650, -1.6650, -1.6650],
          [-1.6824, -1.6824, 

In [154]:
model = initialize_model().cuda()
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=14, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 243407.578125.
Iteration 1, inertia 155710.65625.
Iteration 2, inertia 153367.90625.
Iteration 3, inertia 152571.0.
Iteration 4, inertia 152229.859375.
Iteration 5, inertia 152044.0.
Iteration 6, inertia 151915.140625.
Iteration 7, inertia 151814.265625.
Iteration 8, inertia 151729.296875.
Iteration 9, inertia 151652.453125.
Iteration 10, inertia 151576.96875.
Iteration 11, inertia 151508.734375.
Iteration 12, inertia 151445.21875.
Iteration 13, inertia 151389.1875.
Iteration 14, inertia 151341.875.
Iteration 15, inertia 151299.75.
Iteration 16, inertia 151261.40625.
Iteration 17, inertia 151224.578125.
Iteration 18, inertia 151190.375.
Iteration 19, inertia 151156.96875.
Iteration 20, inertia 151125.859375.
Iteration 21, inertia 151091.171875.
Iteration 22, inertia 151058.03125.
Iteration 23, inertia 151028.40625.
Iteration 24, inertia 151002.046875.
Iteration 25, inertia 150977.125.
Iteration 26, inertia 150953.640625.
Iteration 27, ine

([2.670774059631417,
  2.61784853141867,
  2.562965641097546,
  2.6458109585417056,
  2.4168083644062195],
 [2.59008337970071,
  2.594766236247031,
  2.2608965025261534,
  2.735224747321975,
  2.1212449958066983])

In [155]:
torch.save(model.state_dict(), 'deepcluster_model.pth')

In [78]:
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=14, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 8644.76171875.
Iteration 1, inertia 5951.4013671875.
Iteration 2, inertia 5702.4931640625.
Iteration 3, inertia 5606.3642578125.
Iteration 4, inertia 5553.9541015625.
Iteration 5, inertia 5521.9169921875.
Iteration 6, inertia 5501.322265625.
Iteration 7, inertia 5488.34228515625.
Iteration 8, inertia 5479.9970703125.
Iteration 9, inertia 5474.14306640625.
Iteration 10, inertia 5469.73046875.
Iteration 11, inertia 5466.728515625.
Iteration 12, inertia 5464.5244140625.
Iteration 13, inertia 5462.3359375.
Iteration 14, inertia 5460.4921875.
Iteration 15, inertia 5458.81103515625.
Iteration 16, inertia 5457.365234375.
Iteration 17, inertia 5456.068359375.
Iteration 18, inertia 5454.9345703125.
Iteration 19, inertia 5454.1328125.
Iteration 20, inertia 5453.32470703125.
Iteration 21, inertia 5452.3291015625.
Iteration 22, inertia 5451.3369140625.
Iteration 23, inertia 5450.5751953125.
Iteration 24, inertia 5449.91552734375.
Iteration 25, inerti

([2.6803402127643836,
  2.3968484661822447,
  2.361583117396781,
  2.2705256743479314,
  2.3341632940781616],
 [2.143159402530123,
  2.400692359733147,
  2.3708119761699424,
  1.8542610062010467,
  2.26683363067262])

In [79]:
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=15, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 1039.249755859375.
Iteration 1, inertia 780.84423828125.
Iteration 2, inertia 747.8240966796875.
Iteration 3, inertia 735.0625610351562.
Iteration 4, inertia 725.9406127929688.
Iteration 5, inertia 718.123291015625.
Iteration 6, inertia 711.59912109375.
Iteration 7, inertia 705.9868774414062.
Iteration 8, inertia 701.8674926757812.
Iteration 9, inertia 698.8106079101562.
Iteration 10, inertia 696.4400634765625.
Iteration 11, inertia 694.2005004882812.
Iteration 12, inertia 692.005126953125.
Iteration 13, inertia 690.0314331054688.
Iteration 14, inertia 688.3350830078125.
Iteration 15, inertia 686.9429321289062.
Iteration 16, inertia 685.8116455078125.
Iteration 17, inertia 684.765380859375.
Iteration 18, inertia 683.7196044921875.
Iteration 19, inertia 682.7366943359375.
Iteration 20, inertia 681.7973022460938.
Iteration 21, inertia 680.9169311523438.
Iteration 22, inertia 680.118408203125.
Iteration 23, inertia 679.2962646484375.
Iterati

([2.47920065338247,
  2.3796572418911413,
  2.4623327397730246,
  2.4602977451721366,
  2.5295986932660406],
 [2.253721220889776,
  2.2435940669717986,
  2.4201223356034056,
  2.5638374663159627,
  2.397164034137422])

In [80]:
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=16, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 760.7156982421875.
Iteration 1, inertia 610.7670288085938.
Iteration 2, inertia 596.6004028320312.
Iteration 3, inertia 589.5922241210938.
Iteration 4, inertia 585.9107055664062.
Iteration 5, inertia 584.0486450195312.
Iteration 6, inertia 583.23974609375.
Iteration 7, inertia 582.7196044921875.
Iteration 8, inertia 582.4354248046875.
Iteration 9, inertia 582.213623046875.
Iteration 10, inertia 581.9575805664062.
Iteration 11, inertia 581.6653442382812.
Iteration 12, inertia 581.4273071289062.
Iteration 13, inertia 581.2127685546875.
Iteration 14, inertia 581.0134887695312.
Iteration 15, inertia 580.829833984375.
Iteration 16, inertia 580.6488647460938.
Iteration 17, inertia 580.4718017578125.
Iteration 18, inertia 580.27978515625.
Iteration 19, inertia 580.1484985351562.
Iteration 20, inertia 580.04443359375.
Iteration 21, inertia 579.9447021484375.
Iteration 22, inertia 579.842041015625.
Iteration 23, inertia 579.757080078125.
Iteration

([2.5383481351722064,
  2.580855709184652,
  2.503949740822243,
  2.5344209912142577,
  2.5435519458105547],
 [2.205430351248635,
  2.3678550698491057,
  2.475889388412441,
  2.6955967937895267,
  2.4710066796435313])

In [81]:
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=17, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 1076.631591796875.
Iteration 1, inertia 808.1430053710938.
Iteration 2, inertia 768.0143432617188.
Iteration 3, inertia 747.3872680664062.
Iteration 4, inertia 735.6607055664062.
Iteration 5, inertia 724.4298095703125.
Iteration 6, inertia 716.4656982421875.
Iteration 7, inertia 711.8348999023438.
Iteration 8, inertia 708.4431762695312.
Iteration 9, inertia 704.8443603515625.
Iteration 10, inertia 701.6618041992188.
Iteration 11, inertia 699.6255493164062.
Iteration 12, inertia 698.0443115234375.
Iteration 13, inertia 696.5989990234375.
Iteration 14, inertia 695.5067138671875.
Iteration 15, inertia 694.8052978515625.
Iteration 16, inertia 694.3109130859375.
Iteration 17, inertia 693.883544921875.
Iteration 18, inertia 693.4806518554688.
Iteration 19, inertia 693.1459350585938.
Iteration 20, inertia 692.8790893554688.
Iteration 21, inertia 692.6575927734375.
Iteration 22, inertia 692.4754638671875.
Iteration 23, inertia 692.33203125.
Itera

([2.6278499310417773,
  2.596663354767856,
  2.6457078770337352,
  2.6097028255462646,
  2.612852972707535],
 [2.777938574484649,
  2.671592049283699,
  2.6524501949345063,
  2.448175419979052,
  2.6719940566799365])

In [82]:
train_deepcluster(model, dataloaders['train'], dataloaders['val'], num_clusters=18, num_epochs=5)

[]
Initialization complete
Iteration 0, inertia 831.936767578125.
Iteration 1, inertia 643.7615966796875.
Iteration 2, inertia 627.1077880859375.
Iteration 3, inertia 621.7203979492188.
Iteration 4, inertia 618.4854125976562.
Iteration 5, inertia 616.32421875.
Iteration 6, inertia 614.7935180664062.
Iteration 7, inertia 613.7067260742188.
Iteration 8, inertia 612.8258056640625.
Iteration 9, inertia 612.1923217773438.
Iteration 10, inertia 611.6959228515625.
Iteration 11, inertia 611.255859375.
Iteration 12, inertia 610.8631591796875.
Iteration 13, inertia 610.5299682617188.
Iteration 14, inertia 610.1983642578125.
Iteration 15, inertia 609.955322265625.
Iteration 16, inertia 609.7635498046875.
Iteration 17, inertia 609.5819091796875.
Iteration 18, inertia 609.4307861328125.
Iteration 19, inertia 609.2943725585938.
Iteration 20, inertia 609.17578125.
Iteration 21, inertia 609.0841064453125.
Iteration 22, inertia 608.9785766601562.
Iteration 23, inertia 608.8721313476562.
Iteration 24, i

([2.6856378995309096,
  2.6676709403518224,
  2.645125165367068,
  2.7094157044594542,
  2.725214860353812],
 [2.4307649548341583,
  2.577572619453378,
  2.532918068705496,
  2.637063873655975,
  2.607471105992658])